#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [6]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn
import pandas as pd
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider

## Huangrui's Law School pass bar

In [7]:
class LawSchoolPassBarDataset(StandardDataset):
    """Law School PassBar dataset.

    See https://github.com/microsoft/tempeh for details.
    """

    def __init__(self, label_name='pass_bar', favorable_classes=[1],  
                 protected_attribute_names=['race'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_drop=[],
                 features_to_keep=[],
                 na_values=[], custom_preprocessing=None,
                 metadata=None):
        """See :obj:`RegressionDataset` for a description of the arguments."""
        

        data = pd.read_csv("./Huangrui/law/bar_pass_prediction.csv")
        # data contains 'sex', 'gender', and 'male' which are all identical except for the type;
        data.drop(["sex"], axis=1, inplace=True)
        data.drop(['gender'], axis=1, inplace=True)
        data.rename(columns={'male': 'sex'}, inplace=True)
        #drop grad because every one is not grad
        data.drop(['grad'], axis=1, inplace=True)
        #conver bar1, bar2 to 0/1
        data.bar1 = data.bar1.apply(lambda x: 1 if x == "P" else 0)
        data.bar2 = data.bar2.apply(lambda x: 1 if x == "P" else 0)
        # filter out all records except the ones with the most common two races
        data = data.assign(race=(data["race1"] == "white") * 1)
        data.drop(['race1','race2',"other","asian","black","hisp"], axis=1, inplace=True)
        #drop dnn_bar_pass_prediction, because it is the similar to pass_bar and is not a good data
        data.drop(['dnn_bar_pass_prediction'], axis=1, inplace=True)
        # encode dropout as 0/1
        data = data.assign(dropout=(data["Dropout"] == b"YES") * 1)
        data.drop(['Dropout'], axis=1, inplace=True)
        #drop ID because it is useless
        data.drop(['ID'], axis=1, inplace=True)
        #drop indxgrp indxgrp2 because they are useless
        data.drop(['indxgrp','indxgrp2'], axis=1, inplace=True)
        # drop NaN records for pass_bar
        data = data[(data['pass_bar'] == 1) | (data['pass_bar'] == 0)]
        #drop gpa beacuse it is the same as ugpa
        data.drop(['gpa'], axis=1, inplace=True)
        #drop bar_passed, bar, because they have the same value as pass_bar
        data.drop(['bar_passed','bar'], axis=1, inplace=True)
        # drop NaN records for features
        data = data[np.isfinite(data["lsat"]) & np.isfinite(data['ugpa'])]

        super(LawSchoolPassBarDataset, self).__init__(
            df=data, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)



#### Load dataset and specify options

In [8]:
dataset_orig = LawSchoolPassBarDataset()
privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)
# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05

In [9]:
dataset_orig.feature_names

['decile1b',
 'decile3',
 'decile1',
 'race',
 'cluster',
 'lsat',
 'ugpa',
 'zfygpa',
 'DOB_yr',
 'zgpa',
 'bar1',
 'bar1_yr',
 'bar2',
 'bar2_yr',
 'fulltime',
 'fam_inc',
 'age',
 'parttime',
 'sex',
 'tier',
 'index6040',
 'dropout']

In [10]:
# 5 fold cross validation
Z =  dataset_orig.split(5, shuffle=True,seed = random_seed)
# i th fold
dataset_train1 = Z[0].copy()
dataset_train1.features = np.concatenate((Z[0].features,Z[1].features,Z[2].features,Z[3].features),axis=0)
dataset_train1.scores = np.concatenate((Z[0].scores,Z[1].scores,Z[2].scores,Z[3].scores),axis=0)
dataset_train1.labels = np.concatenate((Z[0].labels,Z[1].labels,Z[2].labels,Z[3].labels),axis=0)
dataset_train1.protected_attributes = np.concatenate((Z[0].protected_attributes,Z[1].protected_attributes,Z[2].protected_attributes,Z[3].protected_attributes),axis=0)
dataset_train1.instance_weights = np.concatenate((Z[0].instance_weights,Z[1].instance_weights,Z[2].instance_weights,Z[3].instance_weights),axis=0)
dataset_train1.instance_names = np.concatenate((Z[0].instance_names,Z[1].instance_names,Z[2].instance_names,Z[3].instance_names),axis=0)
dataset_train1.metadata = Z[0].metadata.copy()
dataset_test1= Z[4].copy()

dataset_train2 = Z[1].copy()
dataset_train2.features = np.concatenate((Z[1].features,Z[2].features,Z[3].features,Z[4].features),axis=0)
dataset_train2.scores = np.concatenate((Z[1].scores,Z[2].scores,Z[3].scores,Z[4].scores),axis=0)
dataset_train2.labels = np.concatenate((Z[1].labels,Z[2].labels,Z[3].labels,Z[4].labels),axis=0)
dataset_train2.protected_attributes = np.concatenate((Z[1].protected_attributes,Z[2].protected_attributes,Z[3].protected_attributes,Z[4].protected_attributes),axis=0)
dataset_train2.instance_weights = np.concatenate((Z[1].instance_weights,Z[2].instance_weights,Z[3].instance_weights,Z[4].instance_weights),axis=0)
dataset_train2.instance_names = np.concatenate((Z[1].instance_names,Z[2].instance_names,Z[3].instance_names,Z[4].instance_names),axis=0)
dataset_train2.metadata = Z[1].metadata.copy()
dataset_test2= Z[0].copy()

dataset_train3 = Z[2].copy()
dataset_train3.features = np.concatenate((Z[2].features,Z[3].features,Z[4].features,Z[0].features),axis=0)
dataset_train3.scores = np.concatenate((Z[2].scores,Z[3].scores,Z[4].scores,Z[0].scores),axis=0)
dataset_train3.labels = np.concatenate((Z[2].labels,Z[3].labels,Z[4].labels,Z[0].labels),axis=0)
dataset_train3.protected_attributes = np.concatenate((Z[2].protected_attributes,Z[3].protected_attributes,Z[4].protected_attributes,Z[0].protected_attributes),axis=0)
dataset_train3.instance_weights = np.concatenate((Z[2].instance_weights,Z[3].instance_weights,Z[4].instance_weights,Z[0].instance_weights),axis=0)
dataset_train3.instance_names = np.concatenate((Z[2].instance_names,Z[3].instance_names,Z[4].instance_names,Z[0].instance_names),axis=0)
dataset_train3.metadata = Z[2].metadata.copy()
dataset_test3= Z[1].copy()

dataset_train4 = Z[3].copy()
dataset_train4.features = np.concatenate((Z[3].features,Z[4].features,Z[0].features,Z[1].features),axis=0)
dataset_train4.scores = np.concatenate((Z[3].scores,Z[4].scores,Z[0].scores,Z[1].scores),axis=0)
dataset_train4.labels = np.concatenate((Z[3].labels,Z[4].labels,Z[0].labels,Z[1].labels),axis=0)
dataset_train4.protected_attributes = np.concatenate((Z[3].protected_attributes,Z[4].protected_attributes,Z[0].protected_attributes,Z[1].protected_attributes),axis=0)
dataset_train4.instance_weights = np.concatenate((Z[3].instance_weights,Z[4].instance_weights,Z[0].instance_weights,Z[1].instance_weights),axis=0)
dataset_train4.instance_names = np.concatenate((Z[3].instance_names,Z[4].instance_names,Z[0].instance_names,Z[1].instance_names),axis=0)
dataset_train4.metadata = Z[3].metadata.copy()
dataset_test4= Z[2].copy()

dataset_train5 = Z[4].copy()
dataset_train5.features = np.concatenate((Z[4].features,Z[0].features,Z[1].features,Z[2].features),axis=0)
dataset_train5.scores = np.concatenate((Z[4].scores,Z[0].scores,Z[1].scores,Z[2].scores),axis=0)
dataset_train5.labels = np.concatenate((Z[4].labels,Z[0].labels,Z[1].labels,Z[2].labels),axis=0)
dataset_train5.protected_attributes = np.concatenate((Z[4].protected_attributes,Z[0].protected_attributes,Z[1].protected_attributes,Z[2].protected_attributes),axis=0)
dataset_train5.instance_weights = np.concatenate((Z[4].instance_weights,Z[0].instance_weights,Z[1].instance_weights,Z[2].instance_weights),axis=0)
dataset_train5.instance_names = np.concatenate((Z[4].instance_names,Z[0].instance_names,Z[1].instance_names,Z[2].instance_names),axis=0)
dataset_train5.metadata = Z[4].metadata.copy()
dataset_test5= Z[3].copy()

#### Save the split dataset

In [11]:
protected_name = "race"
dataset_train1.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_train1.csv".format(protected_name),index=False)
dataset_test1.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_test1.csv".format(protected_name),index=False)
dataset_train2.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_train2.csv".format(protected_name),index=False)
dataset_test2.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_test2.csv".format(protected_name),index=False)
dataset_train3.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_train3.csv".format(protected_name),index=False)
dataset_test3.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_test3.csv".format(protected_name),index=False)
dataset_train4.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_train4.csv".format(protected_name),index=False)
dataset_test4.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_test4.csv".format(protected_name),index=False)
dataset_train5.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_train5.csv".format(protected_name),index=False)
dataset_test5.convert_to_dataframe()[0].to_csv("Huangrui/law/{}/law_test5.csv".format(protected_name),index=False)


#### Split into train, test and validation

In [27]:
dataset_orig_train_valid= dataset_train2.copy()
dataset_orig_test= dataset_test2.copy()

In [28]:
# Get the dataset and split into train and test
dataset_orig_train, dataset_orig_valid = dataset_orig_train_valid.split([0.8], shuffle=True)

#### Clean up training data and display properties of the data

In [29]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(13072, 22)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['race']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['decile1b', 'decile3', 'decile1', 'race', 'cluster', 'lsat', 'ugpa', 'zfygpa', 'DOB_yr', 'zgpa', 'bar1', 'bar1_yr', 'bar2', 'bar2_yr', 'fulltime', 'fam_inc', 'age', 'parttime', 'sex', 'tier', 'index6040', 'dropout']


#### Metric for original training data

In [30]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.116947


### Train classifier on original data

In [31]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

lmod = LogisticRegression()
lmod.fit(X_train, y_train)
y_train_pred = lmod.predict(X_train)

# positive class index
pos_ind = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]

dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred

#### Obtain scores for validation and test sets

In [32]:
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
X_valid = scale_orig.transform(dataset_orig_valid_pred.features)
y_valid = dataset_orig_valid_pred.labels
dataset_orig_valid_pred.scores = lmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = scale_orig.transform(dataset_orig_test_pred.features)
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = lmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

### Find the optimal parameters from the validation set

#### Best threshold for classification only (no fairness)

In [33]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = dataset_orig_valid_pred.scores > class_thresh
    dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
    dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(dataset_orig_valid,
                                             dataset_orig_valid_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    
    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

Best balanced accuracy (no fairness constraints) = 1.0000
Optimal classification threshold (no fairness constraints) = 0.0100


#### Estimate optimal parameters for the ROC method

In [34]:
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

In [35]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.0100
Optimal ROC margin = 0.0000


### Predictions from Validation Set

In [36]:
# Metrics for the test set
fav_inds = dataset_orig_valid_pred.scores > best_class_thresh
dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label

display(Markdown("#### Validation set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_valid_bef = compute_metrics(dataset_orig_valid, dataset_orig_valid_pred, 
                unprivileged_groups, privileged_groups)

#### Validation set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 1.0000
Statistical parity difference = -0.0954
Disparate impact = 0.9016
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000


In [37]:
# Transform the validation set
dataset_transf_valid_pred = ROC.predict(dataset_orig_valid_pred)

display(Markdown("#### Validation set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_transf_valid_pred, 
                unprivileged_groups, privileged_groups)

#### Validation set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 1.0000
Statistical parity difference = -0.0954
Disparate impact = 0.9016
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000


In [38]:
# Testing: Check if the metric optimized has not become worse
assert np.abs(metric_valid_aft[metric_name]) <= np.abs(metric_valid_bef[metric_name])

### Predictions from Test Set

In [39]:
# Metrics for the test set
fav_inds = dataset_orig_test_pred.scores > best_class_thresh
dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

display(Markdown("#### Test set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 1.0000
Statistical parity difference = -0.1132
Disparate impact = 0.8829
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000


In [40]:
# Metrics for the transformed test set
dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

display(Markdown("#### Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 1.0000
Statistical parity difference = -0.1132
Disparate impact = 0.8829
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000


In [41]:
metric_test_aft["Balanced accuracy"]
metric_test_aft["Equal opportunity difference"]
print("The Error for the test dataset is {:.4}".format(1-metric_test_aft["Balanced accuracy"]))
print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))

The Error for the test dataset is 0.0
The Equal opportunity difference for the test dataset is 0.0
